In [ ]:
!mkdir /content/MIMICData

In [ ]:
!unzip /content/PR.zip -d /content/MIMICData

Archive:  /content/PR.zip
   creating: /content/MIMICData/PR/
  inflating: /content/MIMICData/PR/AI.txt  
  inflating: /content/MIMICData/PR/fichier-test.txt  


In [ ]:
!cat /content/MIMICData/PR/fichier-test.txt

Since the invention of computers or machines, their capability to perform various tasks went
on growing exponentially. Humans have developed the power of computer systems in terms
of their diverse working domains, their increasing speed, and reducing size with respect to
time.
A branch of Computer Science named Artificial Intelligence pursues creating the computers or
machines as intelligent as human beings.
According to the father of Artificial Intelligence John McCarthy, it is “The science and
engineering of making intelligent machines, especially intelligent computer programs”.
Artificial Intelligence is a way of making a computer, a computer-controlled robot, or a
software think intelligently, in the similar manner the intelligent humans think.
AI is accomplished by studying how human brain thinks, and how humans learn, decide, and
work while trying to solve a problem, and then using the outcomes of this study as a basis of
developing intelligent software and systems.
Artificial in

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-cvqs2mlz/farm-haystack_44ef3d38589b4cbe8e94d88664c35d2d
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-cvqs2mlz/farm-haystack_44ef3d38589b4cbe8e94d88664c35d2d
  Resolved https://github.com/deepset-ai/haystack.git to commit 4cbc8550d64bc4c3b92998662f125b9c242e4dbe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [ ]:
doc_dir = "/content/MIMICData"


# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)


INFO:haystack.utils.preprocessing:Converting /content/MIMICData/PR/fichier-test.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/PR/AI.txt


In [ ]:
print(docs[:1])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

[<Document: {'content': 'Since the invention of computers or machines, their capability to perform various tasks went\non growing exponentially. Humans have developed the power of computer systems in terms\nof their diverse working domains, their increasing speed, and reducing size with respect to\nA branch of Computer Science named Artificial Intelligence pursues creating the computers or\nmachines as intelligent as human beings.\nAccording to the father of Artificial Intelligence John McCarthy, it is “The science and\nengineering of making intelligent machines, especially intelligent computer programs”.\nArtificial Intelligence is a way of making a computer, a computer-controlled robot, or a\nsoftware think intelligently, in the similar manner the intelligent humans think.\nAI is accomplished by studying how human brain thinks, and how humans learn, decide, and\nwork while trying to solve a problem, and then using the outcomes of this study as a basis of\ndeveloping intelligent softw

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="artificial intelligence?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [ ]:
from pprint import pprint

pprint(prediction['answers'])

[<Answer {'answer': 'Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence', 'type': 'extractive', 'score': 0.12307631969451904, 'context': 'Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence', 'offsets_in_document': [{'start': 0, 'end': 182}], 'offsets_in_context': [{'start': 0, 'end': 182}], 'document_id': '15b3f9b9900b37443010984e28441d30', 'meta': {'name': 'AI.txt'}}>,
 <Answer {'answer': 'machines as intelligent as human beings', 'type': 'extractive', 'score': 0.11467480659484863, 'context': 'tificial Intelligence pursues creating the computers or\nmachines as intelligent as human beings.\nAccording to the father of Artificial Intelligence Jo', 'offsets_in_document': [{'start': 365, 'end': 404}], 'offsets_in_context': [{'sta

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever)
query = 'AI '
result = pipeline.run(query, params={"Retriever": {"top_k": 10}})


In [ ]:
from haystack.utils import print_documents

print_documents(result, max_text_len=100, print_name=True, print_meta=True)


Query: AI 

{   'content': 'Artificial intelligence (AI) is a wide-ranging branch of '
               'computer science concerned with building sm...',
    'meta': {'name': 'AI.txt'},
    'name': 'AI.txt'}

{   'content': 'Since the invention of computers or machines, their capability '
               'to perform various tasks went\n'
               'on grow...',
    'meta': {'name': 'fichier-test.txt'},
    'name': 'fichier-test.txt'}



In [ ]:
[x.to_dict() for x in result["documents"]]

[{'content': "Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence. AI is an interdisciplinary science with multiple approaches, but advancements in machine learning and deep learning are creating a paradigm shift in virtually every sector of the tech industry. \nArtificial intelligence allows machines to model, and even improve upon, the capabilities of the human mind. \nFrom the development of self-driving cars to the proliferation of smart assistants like Siri and Alexa, AI is a growing part of everyday life. As a result, many tech companies across various industries are investing in artificially intelligent technologies.\nLess than a decade after helping the Allied forces win World War II by breaking the Nazi encryption machine Enigma, mathematician Alan Turing changed history a second time with a simple question: “Can machines think?” \nTuring’s 1950 pa